In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from numpy import cos
from math import log
from scipy.interpolate import interp1d
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

#### As equações do modelo:

$dShdt = \mu_H N -a  k b_2  (I_M / N)  S_H - \mu_H S_H\\
dIhdt = a  k b_2  (I_M / N)  S_H - \gamma  I_H - \mu_H I_H\\
dRhdt = \gamma  I_H - \mu_H R_H$

$dSmdt = b - a  k b_1  (I_H / N)  S_M - \mu S_M \\
dEmdt = a k b_1  (I_H / N)  S_M - \mu E_M - b_3 E_M - l E_M\\
dImdt = b_3  E_M  - \mu I_M$

In [2]:
%display typeset

### Analisando o SIR:

In [3]:
var('S_H I_H R_H a b2 k I_M N gamma mu_H')

(S_H, I_H, R_H, a, b2, k, I_M, N, gamma, mu_H)

In [4]:
dShdt = mu_H * N -a * k * b2 * (I_M / N) * S_H - mu_H * S_H
dIhdt = a * k * b2 * (I_M / N) * S_H - gamma * I_H - mu_H * I_H
dRhdt = gamma * I_H - mu_H * R_H

#### Calculando o equilíbrio do SIR

In [5]:
solve([dShdt, dIhdt, dRhdt],[S_H, I_H, R_H])

[[S_H == N^2*mu_H/(I_M*a*b2*k + N*mu_H), I_H == I_M*N*a*b2*k*mu_H/(I_M*a*b2*gamma*k + N*mu_H^2 + (I_M*a*b2*k + N*gamma)*mu_H), R_H == I_M*N*a*b2*gamma*k/(I_M*a*b2*gamma*k + N*mu_H^2 + (I_M*a*b2*k + N*gamma)*mu_H)]]

#### Calculando a Jacobiana do SIR

In [6]:
jack_sir=jacobian([dShdt, dIhdt, dRhdt],[S_H, I_H, R_H])
jack_sir

[-I_M*a*b2*k/N - mu_H                    0                    0]
[        I_M*a*b2*k/N        -gamma - mu_H                    0]
[                   0                gamma                -mu_H]

In [7]:
cp_sir = jack_sir.characteristic_polynomial()
cp_sir

x^3 + (I_M*a*b2*k/N + gamma + 3*mu_H)*x^2 + (I_M*a*b2*gamma*k/N + 2*I_M*a*b2*k*mu_H/N + 2*gamma*mu_H + 3*mu_H^2)*x + I_M*a*b2*gamma*k*mu_H/N + I_M*a*b2*k*mu_H^2/N + gamma*mu_H^2 + mu_H^3

In [8]:
jack_sir.eigenvalues()

[-(I_M*a*b2*k + N*mu_H)/N, -gamma - mu_H, -mu_H]

### Analisando o SEI:

In [9]:
var('S_M E_M I_M b a b1 k I_H N mu b3 l')

(S_M, E_M, I_M, b, a, b1, k, I_H, N, mu, b3, l)

In [10]:
dSmdt = b - a * k * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * k * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M - l * E_M
dImdt = b3 * E_M - mu * I_M

#### Calculando o equilíbrio do SEI

In [11]:
solve([dSmdt, dEmdt, dImdt],[S_M, E_M, I_M])

[[S_M == N*b/(I_H*a*b1*k + N*mu), E_M == I_H*a*b*b1*k/(I_H*a*b1*(b3 + l)*k + N*mu^2 + (I_H*a*b1*k + N*(b3 + l))*mu), I_M == I_H*a*b*b1*b3*k/(I_H*a*b1*(b3 + l)*k*mu + N*mu^3 + (I_H*a*b1*k + N*(b3 + l))*mu^2)]]

#### Calculando a Jacobiana do SEI

In [12]:
jack_sei=jacobian([dSmdt,dEmdt,dImdt],[S_M,E_M,I_M])
jack_sei

[-I_H*a*b1*k/N - mu                  0                  0]
[      I_H*a*b1*k/N       -b3 - l - mu                  0]
[                 0                 b3                -mu]

In [13]:
cp_sei = jack_sei.characteristic_polynomial()
cp_sei

x^3 + (I_H*a*b1*k/N + b3 + l + 3*mu)*x^2 + (I_H*a*b1*b3*k/N + I_H*a*b1*k*l/N + 2*I_H*a*b1*k*mu/N + 2*b3*mu + 2*l*mu + 3*mu^2)*x + I_H*a*b1*b3*k*mu/N + I_H*a*b1*k*l*mu/N + I_H*a*b1*k*mu^2/N + b3*mu^2 + l*mu^2 + mu^3

In [14]:
jack_sei.eigenvalues()

[-(I_H*a*b1*k + N*mu)/N, -b3 - l - mu, -mu]

### Analisando o SIR/SEI:

In [15]:
var('S_H I_H R_H S_M E_M I_M b a b1 b2 k gamma N mu b3 l mu_H')

(S_H, I_H, R_H, S_M, E_M, I_M, b, a, b1, b2, k, gamma, N, mu, b3, l, mu_H)

In [16]:
dShdt = mu_H * N -a * k * b2 * (I_M / N) * S_H - mu_H * S_H
dIhdt = a * k * b2 * (I_M / N) * S_H - gamma * I_H - mu_H * I_H
dRhdt = gamma * I_H - mu_H * R_H
dSmdt = b - a * k * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * k * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M - l * E_M
dImdt = b3 * E_M - mu * I_M

#### Calculando o equilíbrio do SIR/SEI

In [17]:
solve([dShdt, dIhdt, dRhdt, dSmdt, dEmdt, dImdt],[S_H, I_H, R_H, S_M, E_M, I_M])

[[S_H == N, I_H == 0, R_H == 0, S_M == b/mu, E_M == 0, I_M == 0], [S_H == (N^2*gamma*mu^3 + (N^2*b3*gamma + N^2*gamma*l)*mu^2 + (N^2*mu^3 + (N^2*a*b1*k + N^2*b3 + N^2*l)*mu^2 + (N^2*a*b1*b3*k + N^2*a*b1*k*l)*mu)*mu_H)/(a^2*b*b1*b2*b3*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H), I_H == -((N^2*mu^3 + (N^2*b3 + N^2*l)*mu^2)*mu_H^2 - (N*a^2*b*b1*b2*b3*k^2 - N^2*gamma*mu^3 - (N^2*b3*gamma + N^2*gamma*l)*mu^2)*mu_H)/(a^2*b*b1*b2*b3*gamma*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H^2 + (a^2*b*b1*b2*b3*k^2 + N*a*b1*gamma*k*mu^2 + (N*a*b1*b3*gamma*k + N*a*b1*gamma*k*l)*mu)*mu_H), R_H == (N*a^2*b*b1*b2*b3*gamma*k^2 - N^2*gamma^2*mu^3 - (N^2*b3*gamma^2 + N^2*gamma^2*l)*mu^2 - (N^2*gamma*mu^3 + (N^2*b3*gamma + N^2*gamma*l)*mu^2)*mu_H)/(a^2*b*b1*b2*b3*gamma*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H^2 + (a^2*b*b1*b2*b3*k^2 + N*a*b1*gamma*k*mu^2 + (N*a*b1*b3*gamma*k + N*a*b1*gamma*k*l)*mu)*mu_H), S_M == (a*b*b2*b3*gamma*k + (N*mu^2 + (N*b3 + N*l)*mu)*mu_H^2 + (a*b*b2*b3*k + N*gamma*mu^2 + (N*b3*gamma + N*gamma*l)*mu)*mu_H)/(a*b2*b3*gamma*k*mu + (a^2*b1*b2*b3*k^2 + a*b2*b3*k*mu)*mu_H), E_M == -((N*mu^3 + (N*b3 + N*l)*mu^2)*mu_H^2 - (a^2*b*b1*b2*b3*k^2 - N*gamma*mu^3 - (N*b3*gamma + N*gamma*l)*mu^2)*mu_H)/(a*b2*b3*gamma*k*mu^2 + (a*b2*b3^2*gamma*k + a*b2*b3*gamma*k*l)*mu + (a^2*b1*b2*b3^2*k^2 + a^2*b1*b2*b3*k^2*l + a*b2*b3*k*mu^2 + (a^2*b1*b2*b3*k^2 + a*b2*b3^2*k + a*b2*b3*k*l)*mu)*mu_H), I_M == -((N*mu^3 + (N*b3 + N*l)*mu^2)*mu_H^2 - (a^2*b*b1*b2*b3*k^2 - N*gamma*mu^3 - (N*b3*gamma + N*gamma*l)*mu^2)*mu_H)/(a*b2*gamma*k*mu^3 + (a*b2*b3*gamma*k + a*b2*gamma*k*l)*mu^2 + (a*b2*k*mu^3 + (a^2*b1*b2*k^2 + a*b2*b3*k + a*b2*k*l)*mu^2 + (a^2*b1*b2*b3*k^2 + a^2*b1*b2*k^2*l)*mu)*mu_H)]]

#### Calculando a Jacobiana do SIR/SEI

In [18]:
jack=jacobian([dShdt,dIhdt, dRhdt, dSmdt,dEmdt,dImdt],[S_H, I_H, R_H, S_M, E_M, I_M])
jack

[-I_M*a*b2*k/N - mu_H                    0                    0                    0                    0        -S_H*a*b2*k/N]
[        I_M*a*b2*k/N        -gamma - mu_H                    0                    0                    0         S_H*a*b2*k/N]
[                   0                gamma                -mu_H                    0                    0                    0]
[                   0        -S_M*a*b1*k/N                    0   -I_H*a*b1*k/N - mu                    0                    0]
[                   0         S_M*a*b1*k/N                    0         I_H*a*b1*k/N         -b3 - l - mu                    0]
[                   0                    0                    0                    0                   b3                  -mu]

In [19]:
cp = jack.characteristic_polynomial()
cp

x^6 + (I_H*a*b1*k/N + I_M*a*b2*k/N + b3 + gamma + l + 3*mu + 3*mu_H)*x^5 + (I_H*I_M*a^2*b1*b2*k^2/N^2 + I_H*a*b1*b3*k/N + I_M*a*b2*b3*k/N + I_H*a*b1*gamma*k/N + I_M*a*b2*gamma*k/N + I_H*a*b1*k*l/N + I_M*a*b2*k*l/N + 2*I_H*a*b1*k*mu/N + 3*I_M*a*b2*k*mu/N + 3*I_H*a*b1*k*mu_H/N + 2*I_M*a*b2*k*mu_H/N + b3*gamma + gamma*l + 2*b3*mu + 3*gamma*mu + 2*l*mu + 3*mu^2 + 3*b3*mu_H + 2*gamma*mu_H + 3*l*mu_H + 9*mu*mu_H + 3*mu_H^2)*x^4 + (I_H*I_M*a^2*b1*b2*b3*k^2/N^2 - S_H*S_M*a^2*b1*b2*b3*k^2/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2/N^2 + I_H*I_M*a^2*b1*b2*k^2*l/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*mu/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*mu_H/N^2 + I_H*a*b1*b3*gamma*k/N + I_M*a*b2*b3*gamma*k/N + I_H*a*b1*gamma*k*l/N + I_M*a*b2*gamma*k*l/N + I_H*a*b1*b3*k*mu/N + 2*I_M*a*b2*b3*k*mu/N + 2*I_H*a*b1*gamma*k*mu/N + 3*I_M*a*b2*gamma*k*mu/N + I_H*a*b1*k*l*mu/N + 2*I_M*a*b2*k*l*mu/N + I_H*a*b1*k*mu^2/N + 3*I_M*a*b2*k*mu^2/N + 3*I_H*a*b1*b3*k*mu_H/N + 2*I_M*a*b2*b3*k*mu_H/N + 2*I_H*a*b1*gamma*k*mu_H/N + I_M*a*b2*gamma*k*mu_H/N + 3*I_H*a*b1*k*l*mu_H/N + 2*I_M*a*b2*k*l*mu_H/N + 6*I_H*a*b1*k*mu*mu_H/N + 6*I_M*a*b2*k*mu*mu_H/N + 3*I_H*a*b1*k*mu_H^2/N + I_M*a*b2*k*mu_H^2/N + 2*b3*gamma*mu + 2*gamma*l*mu + b3*mu^2 + 3*gamma*mu^2 + l*mu^2 + mu^3 + 2*b3*gamma*mu_H + 2*gamma*l*mu_H + 6*b3*mu*mu_H + 6*gamma*mu*mu_H + 6*l*mu*mu_H + 9*mu^2*mu_H + 3*b3*mu_H^2 + gamma*mu_H^2 + 3*l*mu_H^2 + 9*mu*mu_H^2 + mu_H^3)*x^3 + (I_H*I_M*a^2*b1*b2*b3*gamma*k^2/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2*l/N^2 + I_H*I_M*a^2*b1*b2*b3*k^2*mu/N^2 - S_H*S_M*a^2*b1*b2*b3*k^2*mu/N^2 + 2*I_H*I_M*a^2*b1*b2*gamma*k^2*mu/N^2 + I_H*I_M*a^2*b1*b2*k^2*l*mu/N^2 + I_H*I_M*a^2*b1*b2*k^2*mu^2/N^2 + 2*I_H*I_M*a^2*b1*b2*b3*k^2*mu_H/N^2 - 2*S_H*S_M*a^2*b1*b2*b3*k^2*mu_H/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2*mu_H/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*l*mu_H/N^2 + 4*I_H*I_M*a^2*b1*b2*k^2*mu*mu_H/N^2 + I_H*I_M*a^2*b1*b2*k^2*mu_H^2/N^2 + I_H*a*b1*b3*gamma*k*mu/N + 2*I_M*a*b2*b3*gamma*k*mu/N + I_H*a*b1*gamma*k*l*mu/N + 2*I_M*a*b2*gamma*k*l*mu/N + I_M*a*b2*b3*k*mu^2/N + I_H*a*b1*gamma*k*mu^2/N + 3*I_M*a*b2*gamma*k*mu^2/N + I_M*a*b2*k*l*mu^2/N + I_M*a*b2*k*mu^3/N + 2*I_H*a*b1*b3*gamma*k*mu_H/N + I_M*a*b2*b3*gamma*k*mu_H/N + 2*I_H*a*b1*gamma*k*l*mu_H/N + I_M*a*b2*gamma*k*l*mu_H/N + 3*I_H*a*b1*b3*k*mu*mu_H/N + 4*I_M*a*b2*b3*k*mu*mu_H/N + 4*I_H*a*b1*gamma*k*mu*mu_H/N + 3*I_M*a*b2*gamma*k*mu*mu_H/N + 3*I_H*a*b1*k*l*mu*mu_H/N + 4*I_M*a*b2*k*l*mu*mu_H/N + 3*I_H*a*b1*k*mu^2*mu_H/N + 6*I_M*a*b2*k*mu^2*mu_H/N + 3*I_H*a*b1*b3*k*mu_H^2/N + I_M*a*b2*b3*k*mu_H^2/N + I_H*a*b1*gamma*k*mu_H^2/N + 3*I_H*a*b1*k*l*mu_H^2/N + I_M*a*b2*k*l*mu_H^2/N + 6*I_H*a*b1*k*mu*mu_H^2/N + 3*I_M*a*b2*k*mu*mu_H^2/N + I_H*a*b1*k*mu_H^3/N + b3*gamma*mu^2 + gamma*l*mu^2 + gamma*mu^3 + 4*b3*gamma*mu*mu_H + 4*gamma*l*mu*mu_H + 3*b3*mu^2*mu_H + 6*gamma*mu^2*mu_H + 3*l*mu^2*mu_H + 3*mu^3*mu_H + b3*gamma*mu_H^2 + gamma*l*mu_H^2 + 6*b3*mu*mu_H^2 + 3*gamma*mu*mu_H^2 + 6*l*mu*mu_H^2 + 9*mu^2*mu_H^2 + b3*mu_H^3 + l*mu_H^3 + 3*mu*mu_H^3)*x^2 + (I_H*I_M*a^2*b1*b2*b3*gamma*k^2*mu/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2*l*mu/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2*mu^2/N^2 + I_H*I_M*a^2*b1*b2*b3*gamma*k^2*mu_H/N^2 + I_H*I_M*a^2*b1*b2*gamma*k^2*l*mu_H/N^2 + 2*I_H*I_M*a^2*b1*b2*b3*k^2*mu*mu_H/N^2 - 2*S_H*S_M*a^2*b1*b2*b3*k^2*mu*mu_H/N^2 + 2*I_H*I_M*a^2*b1*b2*gamma*k^2*mu*mu_H/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*l*mu*mu_H/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*mu^2*mu_H/N^2 + I_H*I_M*a^2*b1*b2*b3*k^2*mu_H^2/N^2 - S_H*S_M*a^2*b1*b2*b3*k^2*mu_H^2/N^2 + I_H*I_M*a^2*b1*b2*k^2*l*mu_H^2/N^2 + 2*I_H*I_M*a^2*b1*b2*k^2*mu*mu_H^2/N^2 + I_M*a*b2*b3*gamma*k*mu^2/N + I_M*a*b2*gamma*k*l*mu^2/N + I_M*a*b2*gamma*k*mu^3/N + 2*I_H*a*b1*b3*gamma*k*mu*mu_H/N + 2*I_M*a*b2*b3*gamma*k*mu*mu_H/N + 2*I_H*a*b1*gamma*k*l*mu*mu_H/N + 2*I_M*a*b2*gamma*k*l*mu*mu_H/N + 2*I_M*a*b2*b3*k*mu^2*mu_H/N + 2*I_H*a*b1*gamma*k*mu^2*mu_H/N + 3*I_M*a*b2*gamma*k*mu^2*mu_H/N + 2*I_M*a*b2*k*l*mu^2*mu_H/N + 2*I_M*a*b2*k*mu^3*mu_H/N + I_H*a*b1*b3*gamma*k*mu_H^2/N + I_H*a*b1*gamma*k*l*mu_H^2/N + 3*I_H*a*b1*b3*k*mu*mu_H^2/N + 2*I_M*a*b2*b3*k*mu*mu_H^2/N + 2*I_H*a*b1*gamma*

In [20]:
# Código está abortando
#jack.eigenvalues()

# Internal Python error in the inspect module.

$[-\dfrac{I_M a b_2 k}{N} -\mu_H, -\gamma -\mu_H, -\mu_H, -\dfrac{I_H a b_1 k + N\mu}{N}, -b_3-l-\mu, -\mu]$

### Calculando $\mathcal{R}_0$:

Usando o método proposto por P. van den Driessche no seguinte artigo, e indicado pelo professor Flavio na aula 9 de Modelagem de Fenômenos Biológicos: 

[Reproduction numbers and sub-threshold endemic equilibria for compartmental models of disease transmission](https://pdfs.semanticscholar.org/3cf7/1968a86800215b4e129ec3eda67520832cf0.pdf)

[Aula 9](https://github.com/fccoelho/Modelagem-Matematica-IV/blob/master/Planilhas%20Sage/Aula%209%20-%20Calculo%20do%20R0.ipynb)


Seja $X=(x_1,\ldots, x_n)^T$, tal que $x_i\geq 0$ seja o número de indivíduos em cada compartimento.

Definindo $X_s$ como o conjunto de todos os estados livres de doença.

$$X_s=\{x \geq 0|x_i=0, i=1,\ldots,m\}$$

Supõe-se que cada função é continuamente diferenciável pelo menos duas vezes ($C^2$) em cada variável. As equações são reordenadas para que as $m$ primeiras equações sejam as que contém infectados. Seja ${\cal F}_i(x)$ a taxa de aparecimento de novas infecções no compartimento $i$, ${\cal V}_i^+(x)$ a taxa de entrada de indivíduos no compartimento $i$ por outros meios e ${\cal V}_i^-(x)$ a taxa de saída de indivíduos do compartimento $i$. O modelo de transmissão da doença consiste em condições iniciais não negativas juntamente com o seguinte sistema de equações:

$\dot{x}=f_i(x)={\cal F}_i(x)-{\cal V}_i(x), i=1,\ldots, n$

Onde ${\cal V}_i (x) = {\cal V}_i^{-}(x) - {\cal V}_i^+(x)$ e as funções satisfazem os pressupostos  (A1) - (A5) descritos abaixo. Desde que cada função representa uma transferência dirigida de indivíduos, todos elas são não-negativas.

(A1) Se $x \geq 0 $, então ${\cal F}_i, {\cal V}_i^+, {\cal V}_i ^- \geq 0$ para $i=1, \ldots, n$,

ou seja, se um compartimento estiver vazio, não pode haver saída de indivíduos deste, por morte, infecção ou qualquer outro meio.

(A2) Se $x_i=0$ então ${\cal V}_i^-(x)=0$. Em particular, se $x \in X_s$, então ${\cal V}_i^-(x)=0$ para $i=1,\ldots, m$

(A3) ${\cal F}_i=0$ se $i>m$

(A4) Se $x \in X_S$, então ${\cal F}_i(x) = 0$ e ${\cal V}_i^+(x)=0$ para $i=1,\ldots, m$

(A5) Se ${\cal F}(x)$ é um vetor nulo, então todos os autovalores de $Df(x_0)$ tem parte real negativa.

### ----------------------------------------

Para calcular o $R_0$ é importante distinguir as novas infecções de todas as outras mudanças na população. Nos modelos propostos, os compartimentos que correspondem aos indivíduos infectados são $I$, no caso do SIR, e $E$ e $I$ no caso do SEI. Vamos analisá-los separadamente: 

#### SIR

${\bf m=1}$. A fim de clareza, vamos ordenar os $n=3$ compartimentos da seguinte forma: $[I, S, R]$, separando os $m$ primeiros compartimentos do restante. Podemos também normalizar, de forma a tirar o denominador $N$.

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a  k b_2  I_M  S_H \\
\end{bmatrix} $$

In [21]:
F_cal_sir = matrix([[a*k*b2*I_M*S_H]])
F_cal_sir

[I_M*S_H*a*b2*k]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
I_H(\gamma + \mu_H)\\
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
\end{bmatrix}
\end{equation}
$$

In [22]:
V_cal_neg_sir = matrix([[gamma*I_H + mu_H*I_H]])
V_cal_neg_sir

[I_H*gamma + I_H*mu_H]

In [23]:
V_cal_pos_sir = matrix([[0]])
V_cal_pos_sir

[0]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
I_H(\gamma + \mu_H)\\
\end{bmatrix}
\end{equation}

In [24]:
V_cal_sir = V_cal_neg_sir-V_cal_pos_sir
V_cal_sir

[I_H*gamma + I_H*mu_H]

$$ F = \dfrac{\partial{\cal F}}{\partial I_M} =\begin{bmatrix}
a  k b_2  S_H \\
\end{bmatrix} $$

In [25]:
F_sir = jacobian(F_cal_sir(S_H=1),[I_M])
F_sir

[a*b2*k]

$$ V = \dfrac{\partial{\cal V}}{\partial I_H} =\begin{bmatrix}
\gamma + \mu_H\\
\end{bmatrix} $$

In [26]:
V_sir = jacobian(V_cal_sir(S_H=1),[I_H])
V_sir

[gamma + mu_H]

${\cal R}_0 = \rho (FV^{-1})$

In [27]:
M_sir = F_sir*V_sir.inverse()
M_sir 

[a*b2*k/(gamma + mu_H)]

In [28]:
M_sir.eigenvalues()

[a*b2*k/(gamma + mu_H)]

In [29]:
eigenvalues_sir_list = M_sir.eigenvalues()
eigenvalues_sir_list.sort()

In [30]:
absolute_eigenvalues_sir = np.abs(eigenvalues_sir_list)

# Calcula o maior autovalor, que será o raio espectral
R0_sir = np.max(absolute_eigenvalues_sir)
R0_sir

abs(a*b2*k/(gamma + mu_H))

#### SEI

${\bf m=2}$. A fim de clareza, vamos ordenar os $n=3$ compartimentos da seguinte forma: $[E, I, S]$, separando os $m$ primeiros compartimentos do restante. Vale ressaltar que as transferências dos compartimentos expostos para os infectados não são consideradas novas infecções, mas sim a progressão de um indivíduo infectado através dos vários compartimentos. Novamente removendo $N$, portanto,

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a k b_1 I_H S_M\\
0\\
\end{bmatrix} $$

In [31]:
F_cal_sei = matrix([[a*k*b1*I_H*S_M],[0]])
F_cal_sei

[I_H*S_M*a*b1*k]
[             0]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
E_M (\mu + b_3 + l)\\
\mu I_M
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
b_3 E_M\\
\end{bmatrix}
\end{equation}
$$

In [32]:
V_cal_neg_sei = matrix([[mu * E_M + b3 * E_M + l * E_M], [mu * I_M]])
V_cal_neg_sei

[E_M*b3 + E_M*l + E_M*mu]
[                 I_M*mu]

In [33]:
V_cal_pos_sei = matrix([[0],[b3 * E_M]])
V_cal_pos_sei

[     0]
[E_M*b3]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
E_M (\mu + b_3 + l)\\
\mu I_M - b_3 E_M \\
\end{bmatrix}
\end{equation}

In [34]:
V_cal_sei = V_cal_neg_sei-V_cal_pos_sei
V_cal_sei

[E_M*b3 + E_M*l + E_M*mu]
[       -E_M*b3 + I_M*mu]

Definimos também $F=\left[\frac{\partial {\cal F}_i (x_0)}{\partial x_j}\right]$ e $V=\left[\frac{\partial {\cal V}_i (x_0) }{\partial x_j}\right]$, onde $x_0$ é um DFE (Equilíbrio livre de doença) e $1\leq i,j \leq m$. 

Isto equivale à jacobiana  destas duas matrizes, após substituir $x_0$ ou seja, $S=1$.

$$ F = \dfrac{\partial{\cal F}}{\partial E_M, I_H} =\begin{bmatrix}
\dfrac{\partial akb_1 I_H S_M}{\partial E_M} & \dfrac{\partial akb_1 I_H S_M}{\partial I_H}\\
\dfrac{\partial 0}{\partial E_M} & \dfrac{\partial 0}{\partial I_H}\\
\end{bmatrix} = 
\begin{bmatrix}
0 & akb_1 S_M\\
0 & 0\\
\end{bmatrix}$$

In [35]:
F_sei = jacobian(F_cal_sei(S_M=1),[E_M, I_H])
F_sei

[     0 a*b1*k]
[     0      0]

$$ V = \dfrac{\partial{\cal V}}{\partial E_M, I_M} =\begin{bmatrix}
\dfrac{\partial E_M (\mu + b_3 + l)}{\partial E_M} & \dfrac{\partial E_M (\mu + b_3 + l)}{\partial I_M}\\
\dfrac{\partial \mu I_M - b_3 E_M}{\partial E_M} & \dfrac{\partial \mu I_M - b_3 E_M}{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
\mu+b_3 +l & 0\\
- b_3 & \mu\\
\end{bmatrix}$$

In [36]:
V_sei = jacobian(V_cal_sei(S_M=1),[E_M, I_M])
V_sei

[b3 + l + mu           0]
[        -b3          mu]

${\cal R}_0 = \rho (FV^{-1})$

In [37]:
M_sei = F_sei*V_sei.inverse()
M_sei 

[a*b1*b3*k/((b3 + l + mu)*mu)                    a*b1*k/mu]
[                           0                            0]

In [38]:
M_sei.eigenvalues()

[a*b1*b3*k/((b3 + l)*mu + mu^2), 0]

In [39]:
eigenvalues_sei_list = M_sei.eigenvalues()
eigenvalues_sei_list.sort()

In [40]:
absolute_eigenvalues_sei = np.abs(eigenvalues_sei_list)

# Calcula o maior autovalor, que será o raio espectral
R0_sei = np.max(absolute_eigenvalues_sei)
R0_sei

abs(a*b1*b3*k/((b3 + l)*mu + mu^2))

### Modelo acoplado:

${\bf m=3}$. A fim de clareza, vamos ordenar os $n=6$ compartimentos da seguinte forma: $[I_H, E_M, I_M, S_H, S_M, R_H]$, separando os $m$ primeiros compartimentos do restante. Vale ressaltar que as transferências dos compartimentos expostos para os infectados não são consideradas novas infecções, mas sim a progressão de um indivíduo infectado através dos vários compartimentos. Novamente removendo $N$, portanto,

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a k b_2  I_M  S_H \\
a k b_1 I_H S_M\\
0\\
\end{bmatrix} $$

In [41]:
F_cal = matrix([[a*k*b2*I_M*S_H],[a*k*b1*I_H*S_M],[0]])
F_cal

[I_M*S_H*a*b2*k]
[I_H*S_M*a*b1*k]
[             0]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
I_H(\gamma + \mu_H)\\
E_M (\mu + b_3 + l)\\
\mu I_M
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
0\\
b_3 E_M\\
\end{bmatrix}
\end{equation}
$$

In [42]:
V_cal_neg = matrix([[gamma*I_H +mu_H*I_H],[mu * E_M + b3 * E_M + l * E_M], [mu * I_M]])
V_cal_neg

[   I_H*gamma + I_H*mu_H]
[E_M*b3 + E_M*l + E_M*mu]
[                 I_M*mu]

In [43]:
V_cal_pos = matrix([[0],[0],[b3 * E_M]])
V_cal_pos

[     0]
[     0]
[E_M*b3]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
I_H(\gamma + \mu_H) \\
E_M (\mu + b_3 + l)\\
\mu I_M - b_3 E_M \\
\end{bmatrix}
\end{equation}

In [44]:
V_cal = V_cal_neg-V_cal_pos
V_cal

[   I_H*gamma + I_H*mu_H]
[E_M*b3 + E_M*l + E_M*mu]
[       -E_M*b3 + I_M*mu]

Definimos também $F=\left[\frac{\partial {\cal F}_i (x_0)}{\partial x_j}\right]$ e $V=\left[\frac{\partial {\cal V}_i (x_0) }{\partial x_j}\right]$, onde $x_0$ é um DFE (Equilíbrio livre de doença) e $1\leq i,j \leq m$. 

Isto equivale à jacobiana  destas duas matrizes, após substituir $x_0$ ou seja, $S=1$.

$$ F = \dfrac{\partial{\cal F}}{\partial I_H, E_M, I_M} =\begin{bmatrix}
\dfrac{\partial akb_2 I_M S_H}{\partial I_H} & \dfrac{\partial akb_2 I_M S_H}{\partial E_M} & \dfrac{\partial akb_2 I_M S_H}{\partial I_M}\\
\dfrac{\partial akb_1 I_H S_M}{\partial I_H} & \dfrac{\partial akb_1 I_H S_M}{\partial E_M} & \dfrac{\partial akb_1 I_H S_M}{\partial I_M}\\
\dfrac{\partial 0}{\partial I_H} & \dfrac{\partial 0}{\partial E_M} & \dfrac{\partial 0}{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
0 & 0 & ab_2 S_H\\
ab_1 S_M & 0 & 0\\
0 & 0 & 0
\end{bmatrix}$$

In [45]:
F = jacobian(F_cal(S_H=1, S_M=1),[I_H, E_M, I_M])
F

[     0      0 a*b2*k]
[a*b1*k      0      0]
[     0      0      0]

$$ V = \dfrac{\partial{\cal V}}{\partial I_H, E_M, I_M} =\begin{bmatrix}
\dfrac{\partial I_H(\gamma + \mu_H)}{\partial I_H} & \dfrac{\partial I_H(\gamma + \mu_H)}{\partial E_M} & \dfrac{\partial I_H(\gamma + \mu_H)}{\partial I_M}\\
\dfrac{\partial E_M (\mu + b_3 + l)}{\partial I_H} & \dfrac{\partial E_M (\mu + b_3 + l)}{\partial E_M} & \dfrac{\partial E_M (\mu + b_3 + l)}{\partial I_M}\\
\dfrac{\partial \mu I_M - b_3 E_M }{\partial I_H} & \dfrac{\partial \mu I_M - b_3 E_M }{\partial E_M} & \dfrac{\partial \mu I_M - b_3 E_M }{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
\gamma + \mu_H & 0 & 0\\
0 & b_3 + l+\mu & 0\\
0 & -b_3 & \mu
\end{bmatrix}$$

In [46]:
V = jacobian(V_cal(S_H=1, S_M=1),[I_H, E_M, I_M])
V

[gamma + mu_H            0            0]
[           0  b3 + l + mu            0]
[           0          -b3           mu]

In [47]:
M = F*V.inverse()
M

[                           0 a*b2*b3*k/((b3 + l + mu)*mu)                    a*b2*k/mu]
[       a*b1*k/(gamma + mu_H)                            0                            0]
[                           0                            0                            0]

In [48]:
M.eigenvalues()

[-sqrt(b1*b2*b3/(b3*gamma*mu + gamma*l*mu + gamma*mu^2 + b3*mu*mu_H + l*mu*mu_H + mu^2*mu_H))*a*k,
 sqrt(b1*b2*b3/(b3*gamma*mu + gamma*l*mu + gamma*mu^2 + b3*mu*mu_H + l*mu*mu_H + mu^2*mu_H))*a*k,
 0]

In [49]:
eigenvalues_list = M.eigenvalues()
eigenvalues_list.sort()

In [50]:
absolute_eigenvalues = np.abs(eigenvalues_list)

# Calcula o maior autovalor, que será o raio espectral
R0 = np.max(absolute_eigenvalues)
R0

abs(sqrt(b1*b2*b3/(b3*gamma*mu + gamma*l*mu + gamma*mu^2 + b3*mu*mu_H + l*mu*mu_H + mu^2*mu_H))*a*k)

$\mathcal{R}_0$ do SIR:

$$\Big | \dfrac{ab_2k}{\gamma + \mu_H}\Big |$$

$\mathcal{R}_0$ do SEI:

$$\Big | \dfrac{ab_1b_3k}{(b_3+l+ \mu)\mu}\Big |$$

$\mathcal{R}_0$ acoplado: 

$$\Big |\sqrt{\dfrac{a^2b_1b_2b_3k^2}{\mu(\gamma+\mu_H)(b_3+l+\mu))}} \Big| $$